In [6]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
import pandas as pd
import os
from rasterio.features import shapes
from shapely.geometry import shape

In [3]:
# camada corte

distritos_path = "ShapesProjetoFpolisMapBiomas.gpkg"
layer_distritos = "geometrias_corrigidas"

camada_corte = gpd.read_file(distritos_path, layer=layer_distritos)

In [17]:
# camada raster
# vetoriza

gpkg_saida = "RastersVetorizados_v1.gpkg"

for ano in range(1986,2025):
    raster_path = f"{ano}_31982.tif"
    with rasterio.open(raster_path) as src:
        raster_anual = src.read(1)      # banda 1
        transform = src.transform
        crs = src.crs
        nodata = src.nodata

    geometrias = []
    valores = []
    
    for geom, value in shapes(raster_anual, transform=transform):
        if value is not None:
            geometrias.append(shape(geom))
            valores.append(value)
    
        
    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(
        {"valor": valores},
        geometry=geometrias,
        crs=crs
        )

    # Intersecta
    gdf_intersec = gpd.overlay(
        camada_corte,
        gdf,
        how="intersection"
        )

    # ➕ adiciona o ano
    gdf_intersec["ano"] = ano

    # Nome da camada
    nome_camada = f"{ano}_vetorizado"

    # Exporta (append no mesmo GPKG)
    gdf_intersec.to_file(
        gpkg_saida,
        layer=nome_camada,
        driver="GPKG"
    )

    print(f"Camada {nome_camada} exportada com sucesso.")

Camada 1986_vetorizado exportada com sucesso.
Camada 1987_vetorizado exportada com sucesso.
Camada 1988_vetorizado exportada com sucesso.
Camada 1989_vetorizado exportada com sucesso.
Camada 1990_vetorizado exportada com sucesso.
Camada 1991_vetorizado exportada com sucesso.
Camada 1992_vetorizado exportada com sucesso.
Camada 1993_vetorizado exportada com sucesso.
Camada 1994_vetorizado exportada com sucesso.
Camada 1995_vetorizado exportada com sucesso.
Camada 1996_vetorizado exportada com sucesso.
Camada 1997_vetorizado exportada com sucesso.
Camada 1998_vetorizado exportada com sucesso.
Camada 1999_vetorizado exportada com sucesso.
Camada 2000_vetorizado exportada com sucesso.
Camada 2001_vetorizado exportada com sucesso.
Camada 2002_vetorizado exportada com sucesso.
Camada 2003_vetorizado exportada com sucesso.
Camada 2004_vetorizado exportada com sucesso.
Camada 2005_vetorizado exportada com sucesso.
Camada 2006_vetorizado exportada com sucesso.
Camada 2007_vetorizado exportada c

In [14]:
gdf_intersec.head()

,id,nome,codigo_ibg,area_m2,area_km2,valor,geometry
0,14,SACO DOS LIM�ES,None,7277371.34,7.28,21.0,"POLYGON ((7.43e+05 6.95e+06, 7.43e+05 6.95e+06..."
1,22,SEDE,None,7882128.58,7.88,21.0,"POLYGON ((7.44e+05 6.95e+06, 7.44e+05 6.95e+06..."
2,31,TRINDADE,None,26545689.47,26.55,21.0,"MULTIPOLYGON (((7.43e+05 6.95e+06, 7.43e+05 6...."
3,14,SACO DOS LIM�ES,None,7277371.34,7.28,21.0,"POLYGON ((7.43e+05 6.95e+06, 7.44e+05 6.95e+06..."
4,31,TRINDADE,None,26545689.47,26.55,21.0,"POLYGON ((7.44e+05 6.95e+06, 7.44e+05 6.95e+06..."
